# Composite Risk Engine Validation

This notebook validates the outputs of the **Composite Risk Engine** for our financial statement analysis pipeline.

The **Composite Risk Engine** combines results from:

1. Trend Engine
2. Cash Flow Health Engine
3. Anomaly Efficiency Engine
4. Solvency Engine

Using predefined **weights** and **risk bands**, it calculates a **composite risk score** and assigns a **risk band** (`low`, `medium`, `high`) for each company-year combination.

This notebook will:

- Load financial data
- Generate results from each engine
- Run the composite risk calculation
- Inspect key outputs and contributions
- Provide a clear and reproducible validation workflow


We import:

- **Pandas** for data manipulation.
- **All AFAP engines** required to generate the metrics feeding the composite risk calculation.
- **Default configuration** containing `risk_weights` and `risk_bands`.


In [1]:
# Standard libraries
import pandas as pd
import sys, os

# Add parent directory for engine imports
sys.path.append(os.path.abspath(".."))

# Engine imports
from engines.composite_risk_engine import composite_risk_engine
from engines.ratio_engine_core import ratio_engine
from engines.trend_engine import trend_engine
from engines.cash_flow_engine import cash_flow_engine
from engines.anomaly_efficiency_engine import anomaly_efficiency_engine
from engines.solvency_engine import solvency_engine

# Configuration
from config.defaults import DEFAULT_CLIENT_CONFIG
config = DEFAULT_CLIENT_CONFIG


We use the cleaned financial statements CSV as input. This dataset should include at least:

- `Company` (company name)
- `Year`
- Financial metrics used by the ratio, trend, cash flow, anomaly, and solvency engines


In [2]:
# Load cleaned financial statements
financials = pd.read_csv("../data/cleaned/Kenya_Airways.csv")

# Quick preview
financials.head()


,Company,Year,FS Category,FS Subcategory,Amount
0,Kenya Airways,2021,Assets,Current Assets,"25,685"
1,Kenya Airways,2021,Assets,Non-Current Assets,"129,870"
2,Kenya Airways,2021,Assets,Inventory,"2,152"
3,Kenya Airways,2021,Liabilities,Current Liabilities,"80,965"
4,Kenya Airways,2021,Liabilities,Non-Current Liabilities,"157,927"


We generate results from all engines step by step:

1. **Ratio Engine:** Computes base financial ratios.
2. **Trend Engine:** Evaluates changes in ratios over time.
3. **Cash Flow Engine:** Assesses cash flow health.
4. **Anomaly Efficiency Engine:** Detects unusual financial metrics or patterns.
5. **Solvency Engine:** Checks for potential solvency issues.

All outputs are lists of dictionaries, structured per company-year.


In [3]:
# --- Ratio Engine ---
ratio_results = ratio_engine(financials)
ratios_df = pd.DataFrame(ratio_results)
ratios_df.sort_values(["Company", "Year"], inplace=True)
ratios_df.reset_index(drop=True, inplace=True)

# Flatten metrics once
metrics_df = pd.json_normalize(ratios_df['metrics'])
ratios_flat = pd.concat([ratios_df[['Company', 'Year']], metrics_df], axis=1)

# --- Trend Engine ---
trend_results = trend_engine(ratios_flat)

# --- Cash Flow Engine ---
cash_flow_results = cash_flow_engine(financials)

# --- Anomaly Efficiency Engine ---
anomaly_results = anomaly_efficiency_engine(ratios_flat)

# --- Solvency Engine ---
solvency_results = solvency_engine(ratios_flat)


✅ ratio_engine output validated successfully.
✅ trend_engine output validated successfully.
✅ cash_flow_engine output validated successfully.
✅ anomaly_efficiency_engine output validated successfully.
✅ solvency_engine output validated successfully.


The **Composite Risk Engine**:

- Aggregates individual engine severities using predefined **weights**.
- Computes a **composite score**.
- Assigns a **risk band** based on thresholds defined in `config`.

This gives a consolidated risk view per company-year.


In [4]:
composite_results = composite_risk_engine(
    trend_results,
    cash_flow_results,
    anomaly_results,
    solvency_results,
    config
)

# Convert to DataFrame for inspection
composite_df = pd.DataFrame(composite_results)
composite_df


,Company,Year,composite_score,risk_band
0,Kenya Airways,2024,0.25,low


Before diving into composite scores, we check the structure of each engine’s output.
This ensures consistency of keys and data formats.


In [5]:
def inspect_keys(results, name):
    print(f"\n{name} sample keys:")
    for r in results[:3]:
        print(r.keys())

# Inspect top-level keys in each engine result
inspect_keys(trend_results, "Trend Engine")
inspect_keys(cash_flow_results, "Cash Flow Engine")
inspect_keys(anomaly_results, "Anomaly Engine")
inspect_keys(solvency_results, "Solvency Engine")



Trend Engine sample keys:
dict_keys(['engine', 'Company', 'Year', 'metrics', 'flags', 'severity', 'explanation'])
dict_keys(['engine', 'Company', 'Year', 'metrics', 'flags', 'severity', 'explanation'])
dict_keys(['engine', 'Company', 'Year', 'metrics', 'flags', 'severity', 'explanation'])

Cash Flow Engine sample keys:
dict_keys(['engine', 'Company', 'Year', 'metrics', 'flags', 'severity', 'explanation'])
dict_keys(['engine', 'Company', 'Year', 'metrics', 'flags', 'severity', 'explanation'])
dict_keys(['engine', 'Company', 'Year', 'metrics', 'flags', 'severity', 'explanation'])

Anomaly Engine sample keys:
dict_keys(['engine', 'Company', 'Year', 'metrics', 'flags', 'severity', 'explanation'])
dict_keys(['engine', 'Company', 'Year', 'metrics', 'flags', 'severity', 'explanation'])
dict_keys(['engine', 'Company', 'Year', 'metrics', 'flags', 'severity', 'explanation'])

Solvency Engine sample keys:
dict_keys(['engine', 'Company', 'Year', 'metrics', 'flags', 'severity', 'explanation'])
dic

We inspect individual contributions, e.g., **cash flow severity**, to understand how each component affects the final **composite score**.


In [6]:
# Build a quick lookup for cash flow severities
cash_index = {(r["Company"], r["Year"]): r for r in cash_flow_results}

# Inspect how cash flow severity contributes to composite score
for r in composite_results[:5]:
    key = (r["Company"], r["Year"])
    cash = cash_index.get(key)
    print(
        r["Company"], r["Year"],
        "Composite Score:", r["composite_score"],
        "Risk Band:", r["risk_band"],
        "Cash Severity:", cash.get("severity") if cash else "N/A"
    )


Kenya Airways 2024 Composite Score: 0.25 Risk Band: low Cash Severity: stable


In [7]:
composite_results

[{'Company': 'Kenya Airways',
  'Year': 2024,
  'composite_score': 0.25,
  'risk_band': 'low'}]

# Summary:

- Composite risk scores successfully calculated for all companies and years.
- Individual engine outputs verified and consistent.
- Component contributions (e.g., cash flow, trend, anomaly, solvency) can be inspected per company-year.
- Notebook structured for reproducible validation and audit purposes.

# Next Steps:

1. Expand inspection for all components (trend, anomaly, solvency).
2. Visualize risk bands over years for each company.
3. Integrate automated checks for missing data or engine failures.
4. Optionally, export validated composite results for client reporting.
